In [16]:
%pip install tensorflow tensorflow_hub tensorflow_text
%pip install --upgrade pandas numpy sklearn

  Using cached numpy-1.23.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
Note: you may need to restart the kernel to use updated packages.
Requirement already up-to-date: pandas in /home/m/mekadmini/.local/lib/python3.8/site-packages (2.0.0)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Requirement already up-to-date: sklearn in /home/m/mekadmini/.local/lib/python3.8/site-packages (0.0.post1)
ERROR: tensorflow 2.12.0 has requirement numpy<1.24,>=1.22, but you'll have numpy 1.24.2 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import datetime
import pickle

import os

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

import pickle

In [34]:
def list_saved_models(root_folder='saved_models'):
    model_paths = []
    for dirpath, dirnames, _ in os.walk(root_folder):
        for dirname in dirnames:
            subfolder_path = os.path.join(dirpath, dirname)
            # Only add the directories which start with 'model'
            if dirname.startswith('model'):
                model_paths.append(subfolder_path)
    return model_paths


model_paths = list_saved_models()


In [29]:
# def map_to_entry(model_dir):
#     entries = model_dir.split(sep='/')
#     return {'model_name': entries[1], 'model_parameters': entries[2], 'activation_func': entries[3], 'optimization': entries[4], 'epochs': entries[4], 'date': entries[5]}


# map_to_entry(model_paths[0])


{'model_name': 'small_bert',
 'model_parameters': 'bert_en_uncased_L-2_H-128_A-2',
 'activation_func': 'sigmoid',
 'optimization': 'Adam',
 'epochs': 'Adam',
 'date': '1-epochs'}

In [48]:
def analyze_model(model_dir):
    with open(os.path.join(model_dir, 'testing_data.pickle'), 'rb') as f:
        loaded_obj = pickle.load(f)
    _, x_test, _, y_test = loaded_obj.values()
    model = tf.keras.models.load_model(model_dir)
    test_loss, test_acc, test_prec, test_recall = model.evaluate(
        x_test, y_test)
    entries = model_dir.split(sep='/')

    return {'model_name': entries[1],
            'model_parameters': entries[2],
            'activation_func': entries[3],
            'optimization': entries[4],
            'epochs': entries[5].split('-',1)[0],
            'date': entries[6].split('_',1)[1],
            'test_loss': test_loss,
            'test_acc': test_acc,
            'test_prec': test_prec,
            'test_recall': test_recall}


In [49]:
summary = pd.DataFrame(list(map(analyze_model,model_paths)))
print(summary)

35/35 [==============================] - 3s 66ms/step - loss: 0.0524 - binary_accuracy: 0.9901 - precision: 0.9929 - recall: 0.9329
   model_name               model_parameters activation_func optimization   
0  small_bert  bert_en_uncased_L-2_H-128_A-2         sigmoid         Adam  \
1  small_bert  bert_en_uncased_L-2_H-128_A-2         sigmoid         Adam   
2  small_bert  bert_en_uncased_L-2_H-128_A-2         sigmoid         Adam   

  epochs                 date  test_loss  test_acc  test_prec  test_recall  
0      1  2023-04-04_21-29-41   0.027655  0.991928   0.979452     0.959732  
1      4  2023-04-04_21-54-19   0.044898  0.991928   1.000000     0.939597  
2      5  2023-04-04_21-47-01   0.052441  0.990135   0.992857     0.932886  


In [50]:
summary.to_csv('summary.csv', sep='\t',index=False)

In [51]:
df = pd.read_csv('summary.csv', sep='\t')
df

,model_name,model_parameters,activation_func,optimization,epochs,date,test_loss,test_acc,test_prec,test_recall
0,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,1,2023-04-04_21-29-41,0.027655,0.991928,0.979452,0.959732
1,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,4,2023-04-04_21-54-19,0.044898,0.991928,1.000000,0.939597
2,small_bert,bert_en_uncased_L-2_H-128_A-2,sigmoid,Adam,5,2023-04-04_21-47-01,0.052441,0.990135,0.992857,0.932886
